In [ ]:
!pip install torch transformers accelerator datasets evaluate tqdm pandas nltk scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

## Import Aspect Classification Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ilos-vigil/steam-review-aspect-dataset")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

data-00000-of-00001.arrow:   0%|          | 0.00/529k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/900 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['appid', 'review', 'cleaned_review', 'labels'],
        num_rows: 900
    })
    test: Dataset({
        features: ['appid', 'review', 'cleaned_review', 'labels'],
        num_rows: 200
    })
})

## Load Aspect Classification Model

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

aspect_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
aspect_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 8
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Preprocess Text 1

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

#!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    tokens = [word for word in tokens if word.lower() not in stop_words]

    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Fine-tune Model

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss
import torch
from nltk.tokenize import sent_tokenize
import os

os.environ["WANDB_DISABLED"] = "true"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
aspect_model = aspect_model.to(device)

def preprocess_data(examples):
    examples["review"] = [preprocess_text(review) for review in examples["review"]]
    inputs = aspect_tokenizer(examples["review"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = examples["labels"]  # Dataset has multi-labels for each review
    return inputs

tokenized_datasets = dataset.map(preprocess_data, batched=True)

def compute_metrics(pred):
    logits, labels = pred
    predictions = (logits > 0.5).astype(int)

    # Compute metrics
    precision = precision_score(labels, predictions, average="micro")
    recall = recall_score(labels, predictions, average="micro")
    f1 = f1_score(labels, predictions, average="micro")

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10
)

trainer = Trainer(
    model=aspect_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=aspect_tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-5-ca2eeab2df5b>:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.545200,0.532179,0.781250,0.458716,0.578035
2,0.477000,0.488200,0.779961,0.607034,0.682717
3,0.453800,0.459786,0.801147,0.640673,0.711980
4,0.407200,0.442867,0.832677,0.646789,0.728055
5,0.377900,0.431391,0.833969,0.668196,0.741935
6,0.340600,0.435027,0.827138,0.680428,0.746644
7,0.344800,0.427244,0.818671,0.697248,0.753097
8,0.320600,0.424528,0.829044,0.689602,0.752922


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.545200,0.532179,0.781250,0.458716,0.578035
2,0.477000,0.488200,0.779961,0.607034,0.682717
3,0.453800,0.459786,0.801147,0.640673,0.711980
4,0.407200,0.442867,0.832677,0.646789,0.728055
5,0.377900,0.431391,0.833969,0.668196,0.741935
6,0.340600,0.435027,0.827138,0.680428,0.746644
7,0.344800,0.427244,0.818671,0.697248,0.753097
8,0.320600,0.424528,0.829044,0.689602,0.752922
9,0.278200,0.420214,0.837638,0.694190,0.759197
10,0.299600,0.421400,0.833942,0.698777,0.760399


TrainOutput(global_step=570, training_loss=0.3907776288818895, metrics={'train_runtime': 754.9615, 'train_samples_per_second': 11.921, 'train_steps_per_second': 0.755, 'total_flos': 592031766528000.0, 'train_loss': 0.3907776288818895, 'epoch': 10.0})

## Evaluate Aspect Classification Model

In [ ]:
import torch
import numpy as np

def relaxed_accuracy(y_true, y_pred):
    per_sample_accuracies = [
        sum(y_t == y_p for y_t, y_p in zip(true, pred)) / len(true)
        for true, pred in zip(y_true, y_pred)
    ]
    return sum(per_sample_accuracies) / len(per_sample_accuracies)

# Define aspects and threshold
aspect_labels = ["Recommended", "Story", "Gameplay", "Visual", "Audio", "Technical", "Price", "Suggestions"]
threshold = 0.6

# Prepare ground truth and predictions
ground_truth = []
predictions = []

# Iterate through the dataset to evaluate
for example in dataset["test"]:  # Replace "test" with the relevant split
    # Ground truth for this sample
    ground_truth.append(example["labels"])  # Assuming multi-hot encoding for ground truth labels

    # Model prediction for this sample
    example["review"] = preprocess_text(example["review"])
    inputs = aspect_tokenizer(example["review"], return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = aspect_model(**inputs)
    probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()

    # Convert probabilities to binary predictions
    binary_predictions = (probs > threshold).astype(int)
    predictions.append(binary_predictions)

# Convert to numpy arrays for evaluation
ground_truth = np.array(ground_truth)
predictions = np.array(predictions)

# Compute metrics
relaxed_accuracy_score = relaxed_accuracy(ground_truth, predictions)
precision = precision_score(ground_truth, predictions, average="micro")
recall = recall_score(ground_truth, predictions, average="micro")
f1 = f1_score(ground_truth, predictions, average="micro")
hamming = hamming_loss(ground_truth, predictions)

print(f"Accuracy: {relaxed_accuracy_score:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Hamming Loss: {hamming:.4f}")

Accuracy: 0.8381
Precision: 0.8232
Recall: 0.7691
F1-Score: 0.7953
Hamming Loss: 0.1619


## Aspect Extraction

In [ ]:
review = """The best game ever made by Valve. Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends. You made a great work, Valve, and thank You for this absolute masterpiece. 10/10."""
sentences = sent_tokenize(review)

aspect_labels = ["Recommended", "Story", "Gameplay", "Visual", "Audio", "Technical", "Price", "Suggestions"]
aspect_sentences = {aspect: [] for aspect in aspect_labels}

for sentence in sentences:
    inputs = aspect_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = aspect_model(**inputs)

    probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    for idx, prob in enumerate(probs):
        if prob > 0.6:  # Threshold for relevance
            aspect_sentences[aspect_labels[idx]].append((sentence, prob))

print(aspect_sentences)

{'Recommended': [('The best game ever made by Valve.', 0.92828757), ('Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends.', 0.9535513), ('You made a great work, Valve, and thank You for this absolute masterpiece.', 0.926223), ('10/10.', 0.8802013)], 'Story': [('Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends.', 0.8423036), ('You made a great work, Valve, and thank You for this absolute masterpiece.', 0.6947143)], 'Gameplay': [('Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends.', 0.80955786)], 'Visual': [('Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends.', 0.63107884)], 'Audio': [('Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphe

## Formatting Output

In [ ]:
def concat_aspect_sentences(aspect_sentences):
    new_dict = {}
    for aspect in aspect_sentences:
        if(aspect_sentences[aspect] != []):
            text = ""
            for tuple in aspect_sentences[aspect]:
                text = text + " " + tuple[0]
            new_dict[aspect] = text

    return new_dict

In [ ]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split

review_data = pd.read_csv("/content/reviews.csv")
review_df = pd.DataFrame(review_data).sample(frac=1)[:1000]

# tweet_data = load_dataset("SetFit/tweet_sentiment_extraction")
# tweet_df_train = pd.DataFrame(tweet_data['train'])[:8000]
# tweet_df_test = pd.DataFrame(tweet_data['test'])[:2000]

review_df

,ID,Game,Website,Review,Score
143379,143816,Call of Juarez: Gunslinger,LevelUp,Call of Juarez: Gunslinger offers little chall...,78
102121,102517,Pac-Man Championship Edition 2 Plus,Cubed3,PAC-MAN Championship Edition 2 PLUS could have...,80
294100,295368,Dynasty Warriors: Gundam,Cheat Code Central,The bottom line is if you're smart enough to r...,80
16005,16006,Shin Megami Tensei: Persona 3 FES,GameSpot,This is the definitive version of 2007's best ...,85
190967,191520,PixelJunk Monsters 2,PlayStation LifeStyle,Pixeljunk Monsters 2 does the original great j...,70
...,...,...,...,...,...
104383,104779,Wolfenstein: The New Order,games(TM),Ignore the discordance within its narrative an...,70
22736,22737,DOOM 3,Boomtown,Right out of the box (no patch needed) the gam...,90
6801,6802,Fire Emblem: Awakening,Nintendo Life,Fire Emblem: Awakening's masterful tightrope w...,90
270934,272088,Gunpey DS,GameSpy,Unless you're a hardcore fan of Tetsuya Mizugu...,40


## Reshape Data

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def score_to_sentiment(score):
    if score >= 60:
        return 2
    elif score >= 40:
        return 1
    else:
        return 0

def preprocess_data(text):
    text = lemmatizer.lemmatize(text.lower())
    return text.lower()

# Drop irrelevant columns
# tweet_df_train = tweet_df_train.drop("textID", axis=1)
# tweet_df_test = tweet_df_test.drop("textID", axis=1)
# tweet_df_train = tweet_df_train.drop("label_text", axis=1)
# tweet_df_test = tweet_df_test.drop("label_text", axis=1)
review_df = review_df.drop('Website', axis=1)
review_df = review_df.drop('ID', axis=1)
review_df = review_df.drop('Game', axis=1)
review_df.dropna(inplace=True)

# Rename columns
review_df.rename(columns={"Score": "label"}, inplace=True)
review_df.rename(columns={"Review": "text"}, inplace=True)

# Change values
review_df["label"] = review_df["label"].apply(score_to_sentiment)
review_df["text"] = review_df["text"].apply(preprocess_data)

# positive_df = review_df[review_df['label'] == 2].sample(frac=1)[:500]
# neutral_df = review_df[review_df['label'] == 1].sample(frac=1)[:400]
# negative_df = review_df[review_df['label'] == 0].sample(frac=1)[:350]

# review_df = pd.concat([positive_df, neutral_df, negative_df], axis=0)

# tweet_df_train["text"] = tweet_df_train["text"].apply(preprocess_data)
# tweet_df_test["text"] = tweet_df_test["text"].apply(preprocess_data)

# Save modified
# review_df.to_csv("modified_reviews.csv", index=False)
# tweet_df_train.to_csv("modified_tweets_train.csv", index=False)
# tweet_df_test.to_csv("modified_tweets_test.csv", index=False)

review_df

[nltk_data] Downloading package wordnet to /root/nltk_data...


,text,label
143379,call of juarez: gunslinger offers little chall...,2
102121,pac-man championship edition 2 plus could have...,2
294100,the bottom line is if you're smart enough to r...,2
16005,this is the definitive version of 2007's best ...,2
190967,pixeljunk monsters 2 does the original great j...,2
...,...,...
104383,ignore the discordance within its narrative an...,2
22736,right out of the box (no patch needed) the gam...,2
6801,fire emblem: awakening's masterful tightrope w...,2
270934,unless you're a hardcore fan of tetsuya mizugu...,1


## Initiate Sentiment Classification Model and Tokenizer

In [ ]:
from transformers import XLNetForSequenceClassification, XLNetTokenizer

sentiment_model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=3)
sentiment_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, BertTokenizer
from datasets import Dataset

# Tokenize the dataset
def tokenize_data(examples):
    return sentiment_tokenizer(examples["text"], padding="max_length")

review_df_train, review_df_test = train_test_split(review_df, test_size=0.2, random_state=42)
review_dataset_train = Dataset.from_pandas(review_df_train)
review_dataset_test = Dataset.from_pandas(review_df_test)
# tweet_dataset_train = Dataset.from_pandas(tweet_df_train)
# tweet_dataset_test = Dataset.from_pandas(tweet_df_test)

tokenized_review_train = review_dataset_train.map(tokenize_data, batched=True)
tokenized_review_test = review_dataset_test.map(tokenize_data, batched=True)
# tokenized_tweet_train = tweet_dataset_train.map(tokenize_data, batched=True)
# tokenized_tweet_test = tweet_dataset_test.map(tokenize_data, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

## Fine-tune Model for Sentiment Classification

In [ ]:
from transformers import Trainer, TrainingArguments
import torch
from nltk.tokenize import sent_tokenize
import os

os.environ["WANDB_DISABLED"] = "true"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sentiment_model = sentiment_model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_steps=10,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=1)

    # Compute metrics
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")
    f1 = f1_score(labels, predictions, average="weighted")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model=sentiment_model,
    args=training_args,
    train_dataset=tokenized_review_train,
    eval_dataset=tokenized_review_test,
    compute_metrics=compute_metrics,
    tokenizer=sentiment_tokenizer,
)

trainer.train()

<ipython-input-11-7413eb6f7b6b>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.389700,0.396247,0.860000,0.801895,0.860000,0.828172
2,0.369000,0.381953,0.900000,0.863425,0.900000,0.879856
3,0.230900,0.523228,0.860000,0.801895,0.860000,0.828172
4,0.109700,0.680279,0.875000,0.839293,0.875000,0.856388
5,0.099700,0.608804,0.875000,0.839293,0.875000,0.856388
6,0.081500,0.714703,0.870000,0.858523,0.870000,0.854026
7,0.013900,0.716044,0.865000,0.845641,0.865000,0.855062
8,0.009300,0.749698,0.880000,0.894296,0.880000,0.881536
9,0.001700,0.792142,0.870000,0.871944,0.870000,0.861881
10,0.006000,0.806429,0.880000,0.884011,0.880000,0.876921


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

TrainOutput(global_step=500, training_loss=0.11990466681681573, metrics={'train_runtime': 934.9991, 'train_samples_per_second': 8.556, 'train_steps_per_second': 0.535, 'total_flos': 566026199164800.0, 'train_loss': 0.11990466681681573, 'epoch': 10.0})

## Evaluate Sentiment Classification Model

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, hamming_loss

# Prepare ground truth and predictions
ground_truth = []
predictions = []

# Iterate through the dataset to evaluate
for index, row in review_df_test.iterrows():  # Replace "test" with the relevant split
    # Ground truth for this sample
    ground_truth.append(row["label"])  # Assuming multi-hot encoding for ground truth labels

    # Model prediction for this sample
    inputs = sentiment_tokenizer(row["text"], return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
    # Extract logits and compute predicted class
    logits = outputs.logits  # Assuming your model outputs a `logits` tensor
    predicted_class = torch.argmax(logits, dim=-1).item()  # Get the class index with the highest score

    # Append prediction
    predictions.append(predicted_class)

# Convert to numpy arrays for evaluation
ground_truth = np.array(ground_truth)
predictions = np.array(predictions)
print(ground_truth)
print(predictions)

# Compute metrics
accuracy = accuracy_score(ground_truth, predictions)
precision = precision_score(ground_truth, predictions, average="weighted")
recall = recall_score(ground_truth, predictions, average="weighted")
f1 = f1_score(ground_truth, predictions, average="weighted")
hamming = hamming_loss(ground_truth, predictions)

# Output metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Hamming Loss: {hamming}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 1
 1 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 1 2 2 2 1 2 2 1
 2 2 0 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 1 2
 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 1 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 0 2 2 2 2 2 2 0 2 2 2 2 1]
[2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 1 2 2 2
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 1
 2 2 0 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 1 1 2 2 1 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2
 2 2 0 2 2 2 2 2 2 1 2 2 2 2 2]
Accuracy: 0.88
Precision: 0.884011235955056
Recall: 0.88
F1 Score: 0.8769205346117528
Hamming Loss: 0.12


In [ ]:
review = "It wants to be so many games, and for each of its inspirations, it delivers a different demand. As a logical and unfortunate result, Pokemon Conquest is just too damned demanding."
inputs = sentiment_tokenizer(review, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = sentiment_model(**inputs)

logits = outputs.logits  # Assuming your model outputs a `logits` tensor
predicted_class = torch.argmax(logits, dim=-1).item()  # Get the class index with the highest score

if predicted_class == 2:
  print("Positive")
elif predicted_class == 1:
  print("Neutral")
else:
  print("Negative")

Positive


## Test Full Pipeline

In [ ]:
review = """The best game ever made by Valve. Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends. You made a great work, Valve, and thank You for this absolute masterpiece. 10/10."""
sentences = sent_tokenize(review)

aspect_labels = ["Recommended", "Story", "Gameplay", "Visual", "Audio", "Technical", "Price", "Suggestions"]
aspect_sentences = {aspect: [] for aspect in aspect_labels}

for sentence in sentences:
    inputs = aspect_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = aspect_model(**inputs)

    probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    for idx, prob in enumerate(probs):
        if prob > 0.6:  # Threshold for relevance
            aspect_sentences[aspect_labels[idx]].append((sentence, prob))

review_aspect = concat_aspect_sentences(aspect_sentences)

for aspect in review_aspect:
    aspect_text = review_aspect[aspect]
    inputs = sentiment_tokenizer(aspect_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)

    logits = outputs.logits  # Assuming your model outputs a `logits` tensor
    predicted_class = torch.argmax(logits, dim=-1).item()  # Get the class index with the highest score

    print(aspect, ":", aspect_text)
    if predicted_class == 2:
      print("Positive")
    elif predicted_class == 1:
      print("Neutral")
    else:
      print("Negative")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


## Save Model

In [ ]:
# Define the directory to save the model
save_directory = "/content/drive/MyDrive/NLP"

# Save the model
sentiment_model.save_pretrained(save_directory)

# If using a tokenizer, save it as well
sentiment_tokenizer.save_pretrained(save_directory)

In [ ]:
aspect_model_save_dir = "/content/drive/MyDrive/NLP/AspectModel"

# Save the model
aspect_model.save_pretrained(aspect_model_save_dir)

# If using a tokenizer, save it as well
aspect_model.save_pretrained(aspect_model_save_dir)

In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained(save_directory)
loaded_tokenizer = BertTokenizer.from_pretrained(save_directory)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model = loaded_model.to(device)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
review = """The best game ever made by Valve. Great story, beautiful dialogues with funny jokes, interesting puzzles, perfect atmosphere and a lot of fun in co-op with friends. You made a great work, Valve, and thank You for this absolute masterpiece. 10/10."""
sentences = sent_tokenize(review)

aspect_labels = ["Recommended", "Story", "Gameplay", "Visual", "Audio", "Technical", "Price", "Suggestions"]
aspect_sentences = {aspect: [] for aspect in aspect_labels}

for sentence in sentences:
    inputs = aspect_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = aspect_model(**inputs)

    probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    for idx, prob in enumerate(probs):
        if prob > 0.6:  # Threshold for relevance
            aspect_sentences[aspect_labels[idx]].append((sentence, prob))

review_aspect = concat_aspect_sentences(aspect_sentences)

for aspect in review_aspect:
    aspect_text = review_aspect[aspect]
    inputs = loaded_tokenizer(aspect_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = loaded_model(**inputs)

    logits = outputs.logits  # Assuming your model outputs a `logits` tensor
    predicted_class = torch.argmax(logits, dim=-1).item()  # Get the class index with the highest score

    print(aspect, ":", aspect_text)
    if predicted_class == 2:
      print("Positive")
    elif predicted_class == 1:
      print("Neutral")
    else:
      print("Negative")